In [1]:
import json
import requests
import pandas as pd
from sqlalchemy import create_engine
# from collections import defaultdict

In [2]:
df_wuhan_sex_death = pd.read_csv("wuhan.csv")

In [3]:
bins = [1,10,20,30,40,50,60,70,80,90,100]
sex_groups = df_wuhan_sex_death.groupby(['Sex', pd.cut(df_wuhan_sex_death.Age, bins)])
df_sex_death = sex_groups.size()

In [4]:
df_sex_death = pd.DataFrame(df_sex_death)
df_sex_death = df_sex_death.reset_index(level=[0,1])

In [5]:
new_df_Female_death = df_sex_death.head(4)
new_df_Male_death = df_sex_death.tail(6)
new_df_final_sex_death = pd.merge(new_df_Male_death, new_df_Female_death,on='Age', how="outer")

In [6]:
new_df_final_sex_death['Sex_y'] = "Female"
new_df_final_sex_death['0_y'].fillna(0, inplace=True)
new_df_final_sex_death['Total'] = new_df_final_sex_death['0_x']+new_df_final_sex_death['0_y']
new_df_final_sex_death = new_df_final_sex_death.rename(columns={"Sex_x": "Male", 
                                                                "0_x": "Male_death", 
                                                                "Sex_y": "Female",
                                                                "0_y": "Female_death",
                                                                "Total": "Total_death"})

In [7]:
new_df_final_sex_death = new_df_final_sex_death[['Age','Female','Female_death','Male',"Male_death","Total_death"]]
pd.options.display.float_format = '{:,.0f}'.format

In [8]:
new_df_final_sex_death.index = ['30-40','40-50','50-60','60-70','70-80','80-90']
new_df_final_sex_death = new_df_final_sex_death[['Female','Female_death','Male',"Male_death","Total_death"]]
new_df_final_sex_death = new_df_final_sex_death.reset_index()
new_df_final_sex_death = new_df_final_sex_death.rename(columns={"index": "Age"})
new_df_final_sex_death

,Age,Female,Female_death,Male,Male_death,Total_death
0,30-40,Female,0,Male,2,2
1,40-50,Female,1,Male,1,2
2,50-60,Female,0,Male,4,4
3,60-70,Female,6,Male,9,15
4,70-80,Female,5,Male,8,13
5,80-90,Female,3,Male,9,12


In [9]:
# export df to sqlite
engine = create_engine('sqlite:///wuhan_pneumonia.sqlite')
new_df_final_sex_death.to_sql('Sex_Death', engine, index=False, if_exists='replace')

In [10]:
new_df_final_sex_death = pd.read_sql_query('SELECT * FROM Sex_Death',engine)
new_df_final_sex_death

,Age,Female,Female_death,Male,Male_death,Total_death
0,30-40,Female,0,Male,2,2
1,40-50,Female,1,Male,1,2
2,50-60,Female,0,Male,4,4
3,60-70,Female,6,Male,9,15
4,70-80,Female,5,Male,8,13
5,80-90,Female,3,Male,9,12
